In [14]:
#!pip3 install Pillow
from PIL import Image, ImageDraw,ImageColor
import pandas as pd
import numpy as np
import keras.preprocessing.image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from sklearn.model_selection import train_test_split
import keras.utils as np_utils
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


In [2]:
from keras.models import load_model
from IPython.display import Image, display
import os
#saved_imgs = "./saved"

This is basically so that it runs on gpu

In [3]:
print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

tf.__version__ is 2.2.0
tf.keras.__version__ is: 2.3.0-tf


In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


## READ DATA

In [5]:
#path = '/images/'
dataset = pd.read_excel('rgb2_2.xlsx')
dataset.head(20)

,Color Name,RGB Hex
0,Blue,635688
1,Blue,F0F8FF
2,Blue,8A2BE2
3,Blue,5F9F9F
4,Blue,5F9EA0
5,Blue,5F9EA0
6,Blue,98F5FF
7,Blue,8EE5EE
8,Blue,7AC5CD
9,Blue,53868B


In [6]:
dataset.head(40)

,Color Name,RGB Hex
0,Blue,635688
1,Blue,F0F8FF
2,Blue,8A2BE2
3,Blue,5F9F9F
4,Blue,5F9EA0
5,Blue,5F9EA0
6,Blue,98F5FF
7,Blue,8EE5EE
8,Blue,7AC5CD
9,Blue,53868B


In [7]:
len(dataset['RGB Hex'])

398

In [8]:
dataset['Color Name'].unique()

array(['Blue', 'Green', 'Red'], dtype=object)

In [9]:
dataset = dataset.rename(columns = {'Color Name': 'color', 'RGB Hex': 'Hex'})

In [10]:
dataset

,color,Hex
0,Blue,635688
1,Blue,F0F8FF
2,Blue,8A2BE2
3,Blue,5F9F9F
4,Blue,5F9EA0
...,...,...
393,Green,238E35
394,Green,239E33
395,Green,236E41
396,Green,238E73


In [11]:
dataset['Hex'][0]

635688

## BELOW CELLS ARE AN EXAMPLE OF PRINTING AN IMAGE

In [15]:
img = Image.new('RGB',(28,28),'#' + str(dataset['Hex'][0] ))

In [16]:
print(img_to_array(img)[0,0])

[ 99.  86. 136.]


In [17]:
print(type(img))
print(img.format)
print(img.mode)
print(img.size)
# show the image
#img.show()
display(img)

<class 'PIL.Image.Image'>
None
RGB
(28, 28)


### SINCE ALL PIXEL VALUES ARE SAME ONLY ONE PIXEL VALUE IS ENOUGH FOR PREDICTION

In [18]:
images = []
for i in range(len(dataset.Hex)):
    img = Image.new('RGB',(28,28),'#' + str(dataset['Hex'][i]))
    img_arr = img_to_array(img)[0,0]
    img_arr = img_arr.reshape(1, 1, img_arr.shape[0])
    images.append(img_arr)

In [19]:
images[0].shape

(1, 1, 3)

In [20]:
x = np.concatenate(images, axis = 0)

In [21]:
x.shape

(398, 1, 3)

In [22]:
array_to_img(image_array[156]).show()
print(dataset['color'][156])

NameError: name 'image_array' is not defined

In [23]:
y = dataset['color'].values
#y = y.reshape(y.shape[0],1)

## PREPARE LABELS

In [24]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
print(integer_encoded)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0
 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 1 1 1 1 1 1 2 1 1 1 1 1]


### SAVE LABELS 

In [25]:
labels = label_encoder.classes_
labels

array(['Blue', 'Green', 'Red'], dtype=object)

In [26]:
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)
print(y)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [27]:
y.shape

(398, 3)

### TRAIN-TEST SPLIT 

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, shuffle = True)

In [29]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape, type(y_train[0]))

(318, 1, 3) (80, 1, 3) (318, 3) (80, 3) <class 'numpy.ndarray'>


In [30]:
x_train1 = x_train.reshape(x_train[0], 784, 3)
x_train1.shape

TypeError: only integer scalar arrays can be converted to a scalar index

In [31]:
x_train /= 255
x_test /= 255

##  MODEL DEFINE

In [32]:
model = Sequential()
model.add(Dense(15, activation = 'relu', input_shape=(1,3)))                            
#model.add(Dropout(0.2))

model.add(Dense(15, activation = 'relu'))
#model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(y.shape[1], activation = 'softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1, 15)             60        
_________________________________________________________________
dense_2 (Dense)              (None, 1, 15)             240       
_________________________________________________________________
flatten_1 (Flatten)          (None, 15)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 48        
Total params: 348
Trainable params: 348
Non-trainable params: 0
_________________________________________________________________


In [34]:
history = model.fit(x_train, y_train,
         batch_size=1, epochs=40,
         verbose = 1,
         shuffle = True,
         validation_data=(x_test, y_test)
         )

Train on 318 samples, validate on 80 samples
Epoch 1/40
318/318 [==============================] - 2s 7ms/step - loss: 0.8771 - accuracy: 0.6824 - val_loss: 0.7001 - val_accuracy: 0.8000
Epoch 2/40
318/318 [==============================] - 2s 6ms/step - loss: 0.5985 - accuracy: 0.8145 - val_loss: 0.4905 - val_accuracy: 0.8250
Epoch 3/40
318/318 [==============================] - 2s 6ms/step - loss: 0.4135 - accuracy: 0.8962 - val_loss: 0.3214 - val_accuracy: 0.9250
Epoch 4/40
318/318 [==============================] - 2s 6ms/step - loss: 0.3191 - accuracy: 0.9151 - val_loss: 0.2331 - val_accuracy: 0.9500
Epoch 5/40
318/318 [==============================] - 2s 5ms/step - loss: 0.2701 - accuracy: 0.9245 - val_loss: 0.1990 - val_accuracy: 0.9375
Epoch 6/40
318/318 [==============================] - 2s 6ms/step - loss: 0.2273 - accuracy: 0.9497 - val_loss: 0.1685 - val_accuracy: 0.9750
Epoch 7/40
318/318 [==============================] - 2s 5ms/step - loss: 0.2106 - accuracy: 0.9497 - v

In [35]:
#SAVE MODEL
model.save('model.h5')

In [36]:
#LOAD MODEL
model = load_model('model.h5')

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1, 15)             60        
_________________________________________________________________
dense_2 (Dense)              (None, 1, 15)             240       
_________________________________________________________________
flatten_1 (Flatten)          (None, 15)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 48        
Total params: 348
Trainable params: 348
Non-trainable params: 0
_________________________________________________________________


## TRYING PREDICTIONS WITH SOME OF THE TEST DATA 

In [38]:
test_sample = x_test[57:68]
sample_actual = y_test[57:68]
predict = model.predict_on_batch(test_sample)

In [39]:
predict

array([[2.2770122e-03, 3.8901576e-01, 6.0870725e-01],
       [9.9987781e-01, 1.1368977e-04, 8.4788089e-06],
       [1.5309161e-04, 9.9952066e-01, 3.2618231e-04],
       [8.1962291e-03, 9.4816409e-05, 9.9170893e-01],
       [2.7755855e-04, 1.3983546e-05, 9.9970847e-01],
       [4.0782182e-03, 7.3686479e-06, 9.9591440e-01],
       [1.0006744e-03, 5.3786030e-06, 9.9899393e-01],
       [1.6923102e-04, 9.9981624e-01, 1.4492312e-05],
       [2.2740291e-04, 3.4311429e-07, 9.9977225e-01],
       [2.4831077e-04, 2.1214144e-06, 9.9974960e-01],
       [2.8556325e-03, 4.7903194e-07, 9.9714392e-01]], dtype=float32)

In [40]:
pred_labels = np.argmax(predict, axis=-1)
labels[pred_labels[0]]

'Red'

In [41]:
pred_labels = np.argmax(predict, axis=-1)
actual_labels = np.argmax(sample_actual, axis=-1)

for i in range(len(predict)):
    print('Actual: ' + labels[actual_labels[i]] + ' and Predicted: ' + labels[pred_labels[i]])

Actual: Green and Predicted: Red
Actual: Blue and Predicted: Blue
Actual: Green and Predicted: Green
Actual: Red and Predicted: Red
Actual: Red and Predicted: Red
Actual: Red and Predicted: Red
Actual: Red and Predicted: Red
Actual: Green and Predicted: Green
Actual: Red and Predicted: Red
Actual: Red and Predicted: Red
Actual: Red and Predicted: Red


## MAKE A CUSTOM PREDICTION WITH A HEX VALUE 

In [42]:
def make_prediction(val):
    labels = ['Blue', 'Green', 'Red']
    #PLEASE RUN LOAD_MODEL ABOVE IF NOT RETRAINING MODEL
    img = Image.new('RGB',(28,28),'#' + val)
    img = img_to_array(img)[0,0]
    img= img.reshape(1, 1, img.shape[0])
    val_predict = model.predict(img)
    pred_label = np.argmax(val_predict, axis=-1)
    print("Predicted Color: " + labels[pred_label[0]])

In [43]:
val = input("ENTER HEX: ")
make_prediction(val)

ENTER HEX: 23E67F
Predicted Color: Green


In [44]:
def make_prediction_path(path):
    labels = ['Blue', 'Green', 'Red']
    #PLEASE RUN LOAD_MODEL ABOVE IF NOT RETRAINING MODEL
    #img = Image.new('RGB',(28,28),'#' + val)
    img = load_img(path)
    img = img_to_array(img)[0,0]
    img= img.reshape(1, 1, img.shape[0])
    val_predict = model.predict(img)
    pred_label = np.argmax(val_predict, axis=-1)
    print("Predicted Color: " + labels[pred_label[0]])

In [53]:
path = input("please input the path in double quotes: ")
make_prediction_path(path)

please input the path in double quotes: G:\IP_LP3_MACHINE_LEARNING_SHIVAM_KOTAK_2676\saved\151.jpg
Predicted Color: Green
